# Julia and the Power of Multiple Dispatch

Julia is built around the concept of *multiple dispatch*. Multiple dispatch means that when you call a function 

```juila
f(a, b, c)
```

Julia chooses *which implementation* of the function `f` to call based on the types of `a`, `b`, and `c`. Compare this with Python, in which

```python
# python
f(a, b, c)
```

always calls the same function `f` for any `a`, `b`, and `c`,

or

```python
# python
a.f(b, c)
```

which chooses the version of `f` based on the class of `a`, but not `b` or `c`. 

This allows us to do some really cool things in Julia that would be difficult in other languages. As an example, let's show off by building a simple tool for automatic differentiation. 

By the way, if you want to use automatic differentiation in your own code, the excellent [ForwardDiff.jl](https://github.com/JuliaDiff/ForwardDiff.jl) package provides a complete implementation of the basic idea that we're showing off here. 

# Example: Automatic Differentiation

A common problem in computing (especially in robotics) is:

Given a function $f(x)$, find the function $df(x)$ which evaluates $\frac{d f}{d x}$ at x. 

When $f(x)$ is a mathematical expression, this is straightforward: we just work out the derivative analytically and write it down. But what do we do when $f(x)$ is a function defined in code? 

We have a few options: 

### Analytic differentiation

If we have access to the code of $f(x)$ or the math it was based on, we can work out its derivative by hand and then write code to implement that derivative. This works fine, but it's wasteful (we end up writing code to implement both $f$ and $df$) and error-prone (we have to make sure both pieces of code match...forever). 

### Numerical differentiation

If we don't want to write out $df$ by hand, we can estimate the derivative at some input $x$ by perturbing that input slightly: 

$$
\left. \frac{d f}{dx} \right|_x \approx \frac{f(x + \delta x) - f(x)}{\delta x}
$$

Ideally, this gives us the exact answer as $\delta x \to 0$. Unfortunately, computers are not mathematically ideal. In particular, as $\delta x$ shrinks, the error induced by floating-point rounding becomes more significant. On the other hand, as $\delta x$ grows, our estimate of the derivative becomes less mathematically ideal. 

### Automatic differentiation

Automatic differentiation (or autodiff) refers to a family of methods for computing exact derivatives, generally by running a special data type through your code. 

In forward-mode autodiff (the only kind we'll discuss here), we create a *dual* number, which stores both its value and its derivative. Then we implement basic mathematical operations on that dual number and let the chain rule take care of the rest. 

This is easier to see with an example. 

First, we'll create a structure to represent our dual numbers: 

In [9]:
# The {T} annotation indicates that our `Dual` type is 
# parameterized (i.e. generic). This struct can hold 
# any pair of value and derivative, as long as both are
# of the same type. 

struct Dual{T}
    value::T
    derivative::T
end

# For example, we can construct a `Dual` with Float64 entries:
Dual(1.0, 0.0)

Dual{Float64}(1.0, 0.0)

In [10]:
# or a Dual with integer entries:
Dual(1, 0)

Dual{Int64}(1, 0)

In [11]:
# and Julia will generate specialized code for every new kind of `Dual{T}` we use. 

Now let's implement addition. Julia already knows how to add numbers, we just have to teach it to also add Duals. First, we need to import the `+` function so that we can extend its behavior:

In [12]:
import Base: +  # Base is the common library of tools and functions that ship with Julia

Functions in Julia can have lots of different methods that act on different input types. To see all the existing ways you can use a given function, use `methods()`: 

In [13]:
methods(+)

# 180 methods for generic function "+":
+(x::Bool, z::Complex{Bool}) in Base at complex.jl:232
+(x::Bool, y::Bool) in Base at bool.jl:89
+(x::Bool) in Base at bool.jl:86
+(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:96
+(x::Bool, z::Complex) in Base at complex.jl:239
+(a::Float16, b::Float16) in Base at float.jl:372
+(x::Float32, y::Float32) in Base at float.jl:374
+(x::Float64, y::Float64) in Base at float.jl:375
+(z::Complex{Bool}, x::Bool) in Base at complex.jl:233
+(z::Complex{Bool}, x::Real) in Base at complex.jl:247
+(x::Char, y::Integer) in Base at char.jl:40
+(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:312
+(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:334
+(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:327
+(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:321
+(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:289
+(x::BigInt, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.GMP at gmp.jl:346
+(x::BigInt, c::Union{Int16, Int32, Int64, Int8}) in Base.GMP at gmp.jl:362
+(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:460
+(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:453
+(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:447
+(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:308
+(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:277
+(x::BigFloat, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.MPFR at mpfr.jl:284
+(x::BigFloat, c::Union{Int16, Int32, Int64, Int8}) in Base.MPFR at mpfr.jl:292
+(x::BigFloat, c::Union{Float16, Float32, Float64}) in Base.MPFR at mpfr.jl:300
+(B::BitArray{2}, J::UniformScaling) in Base.LinAlg at linalg/uniformscaling.jl:59
+(a::Base.Pkg.Resolve.VersionWeights.VWPreBuildItem, b::Base.Pkg.Resolve.VersionWeights.VWPreBuildItem) in Base.Pkg.Resolve.VersionWeights at pkg/resolve/versionweight.jl:87
+(a::Base.Pkg.Resolve.VersionWeights.VWPreBuild, b::Base.Pkg.Resolve.VersionWeights.VWPreBuild) in Base.Pkg.Resolve.VersionWeights at pkg/resolve/versionweight.jl:135
+(a::Base.Pkg.Resolve.VersionWeights.VersionWeight, b::Base.Pkg.Resolve.VersionWeights.VersionWeight) in Base.Pkg.Resolve.VersionWeights at pkg/resolve/versionweight.jl:197
+(a::Base.Pkg.Resolve.MaxSum.FieldValues.FieldValue, b::Base.Pkg.Resolve.MaxSum.FieldValues.FieldValue) in Base.Pkg.Resolve.MaxSum.FieldValues at pkg/resolve/fieldvalue.jl:44
+(x::Base.Dates.CompoundPeriod, y::Base.Dates.CompoundPeriod) in Base.Dates at dates/periods.jl:349
+(x::Base.Dates.CompoundPeriod, y::Base.Dates.Period) in Base.Dates at dates/periods.jl:347
+(x::Base.Dates.CompoundPeriod, y::Base.Dates.TimeType) in Base.Dates at dates/periods.jl:387
+(x::Date, y::Base.Dates.Day) in Base.Dates at dates/arithmetic.jl:77
+(x::Date, y::Base.Dates.Week) in Base.Dates at dates/arithmetic.jl:75
+(dt::Date, z::Base.Dates.Month) in Base.Dates at dates/arithmetic.jl:58
+(dt::Date, y::Base.Dates.Year) in Base.Dates at dates/arithmetic.jl:32
+(dt::Date, t::Base.Dates.Time) in Base.Dates at dates/arithmetic.jl:20
+(t::Base.Dates.Time, dt::Date) in Base.Dates at dates/arithmetic.jl:24
+(x::Base.Dates.Time, y::Base.Dates.TimePeriod) in Base.Dates at dates/arithmetic.jl:81
+(dt::DateTime, z::Base.Dates.Month) in Base.Dates at dates/arithmetic.jl:52
+(dt::DateTime, y::Base.Dates.Year) in Base.Dates at dates/arithmetic.jl:28
+(x::DateTime, y::Base.Dates.Period) in Base.Dates at dates/arithmetic.jl:79
+(y::AbstractFloat, x::Bool) in Base at bool.jl:98
+(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:32
+(x::Integer, y::Ptr) in Base at pointer.jl:128
+(z::Complex, w::Complex) in Base at complex.jl:221
+(z::Complex, x::Bool) in Base at complex.jl:240
+(x::Real, z::Complex{Bool}) in Base at complex.jl:246
+(x::Real, z::Complex) in Base at complex.jl:258
+(z::Complex, x::Real) in Base at complex.jl:259
+(x::Rational, y::Rational) in Base at rational.jl:245

We're just going to add a few new methods for `+` to deal with `Dual`s: 

In [14]:
+(n::Number, d::Dual)   = Dual(d.value + n,         d.derivative)
+(d::Dual,   n::Number) = Dual(d.value + n,         d.derivative)
+(d1::Dual,  d2::Dual)  = Dual(d1.value + d2.value, d1.derivative + d2.derivative)

+ (generic function with 183 methods)

Note that `Number` is an abstract type, representing an entire family of types. By defining `+` for `Dual` and `Number`, we get the right behavior for any subtype of `Number`, like `Int`, `Float64`, `UInt8`, `BigInt`, etc.

Now we can add Duals, and they will propagate their derivative information: 

In [15]:
d1 = Dual(1.0, 1.0)

d1 + 5

Dual{Float64}(6.0, 1.0)

In [16]:
d2 = Dual(1.0, 0.5)
d1 + d2

Dual{Float64}(2.0, 1.5)

Let's implement multiplication using the product rule from basic calculus: 

In [31]:
import Base: *
*(n::Number, d::Dual)   = Dual(n * d.value,         d.derivative * n)
*(d::Dual,   n::Number) = Dual(n * d.value,         d.derivative * n)
*(d1::Dual,  d2::Dual)  = Dual(d1.value * d2.value, d1.value * d2.derivative + d2.value * d1.derivative)

* (generic function with 185 methods)

### Aside: Why Julia is Different

We can do this kind of operator overloading in many languages. But the cool part about Julia is that there's nothing special about `+` or other operators. We can overload any function we want for any combination of types. Let's implement a few more functions for our `Dual` type:

In [32]:
import Base: sin, cos

sin(d::Dual) = Dual(sin(d.value),  cos(d.value) * d.derivative)
cos(d::Dual) = Dual(cos(d.value), -sin(d.value) * d.derivative)

cos (generic function with 11 methods)

Now we can use this `Dual` number to perform automatic differentiation! First, we need a function to differentiate:

In [33]:
f(x) = sin(3x + 5)

f (generic function with 1 method)

To evaluate the derivative of `f` at `x`, we first create a Dual with value `x` and derivative $\frac{d x}{d x} = 1$

In [34]:
x = π / 4
x_dual = Dual(x, 1.0)

Dual{Float64}(0.7853981633974483, 1.0)

Now we just pass our `Dual` through the function and check the `derivative` field of the result:

In [35]:
y = f(x_dual)
df = y.derivative
println("The derivative of f at $x is $df")

The derivative of f at 0.7853981633974483 is 1.4324472070543601


We can compare that against numerical differentiation: 

In [36]:
δx = 1e-8
estimate = (f(x + δx) - f(x)) / (δx)
println("The derivative of f at $x is approximately $estimate")

The derivative of f at 0.7853981633974483 is approximately 1.4324471608873068


Success! We got an exact measurement (up to machine accuracy) of the derivative of `f` without any numerical differentation. The best part is that in Julia, this kind of work is extremely efficient: 

In [37]:
using BenchmarkTools

In [38]:
@benchmark f($x)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     12.870 ns (0.00% GC)
  median time:      12.916 ns (0.00% GC)
  mean time:        13.400 ns (0.00% GC)
  maximum time:     116.133 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     999

In [39]:
@benchmark f($x_dual)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     26.873 ns (0.00% GC)
  median time:      27.001 ns (0.00% GC)
  mean time:        28.157 ns (0.00% GC)
  maximum time:     137.186 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     995

There's no overhead introduced by our Dual type: computing `f` *and its derivative* takes only twice as long as computing the value of `f`. 

## More operations

Multiple dispatch is the basis for much of Julia's power and flexibility. We can define new interesting types, give them behaviors, and then use those behaviors to do more complex things. As a very basic demonstration, now that we've defined multiplication and addition for our Dual type, we get sums and exponentiation for free:

In [61]:
d = Dual(0.5, 1.0)
(d + 1)^2

Dual{Float64}(1.5, 1.5)

In [62]:
sum([d, 2 * d, d^2])

Dual{Float64}(2.0, 3.5)

# Completing the Implementation

To make a real autodiff library, we just need to automate the process of creating a new Dual, passing it through a function, and extracting its derivative. Functions in Julia are first-class, so it's easy to create a function that operates on other functions:

In [40]:
function derivative(f, x)
    x_dual = Dual(x, one(x))  # one(x) gives the unit value of whatever x's type is. So, for an integer x, one(x) == 1
    y_dual = f(x_dual)
    y_dual.derivative
end

derivative (generic function with 1 method)

In [41]:
derivative(f, π/4)

1.4324472070543601

In [57]:
f2(x) = x^2
derivative(f2, 2.0)

4.0

In [58]:
# Check against the analytical derivative
@assert derivative(f2, 2.0) ≈ 2 * 2.0

In [59]:
f3(x) = 3x + sin(x^2 * 10) + cos(x)
derivative(f3, 0.5)

-5.49086169407354

In [60]:
# Check against the analytical derivative
@assert derivative(f3, 0.5) ≈ 3 + cos(0.5^2 * 10) * (10 * 2 * 0.5) - sin(0.5)

To get even more fancy, we can make `derivative(f)` return a *new function* `df` which evaluates the derivative of `f`:

In [53]:
derivative(f) = x -> derivative(f, x)  # the -> syntax creates an anonymous function

derivative (generic function with 2 methods)

In [54]:
df = derivative(f)

(::#5) (generic function with 1 method)

In [55]:
df(π/4)

1.4324472070543601

And despite the use of functions-as-data and anonymous functions, we still haven't lost any performance! 

In [56]:
@benchmark $df($x)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     26.492 ns (0.00% GC)
  median time:      26.564 ns (0.00% GC)
  mean time:        27.599 ns (0.00% GC)
  maximum time:     56.583 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     995